# Functions to scrape data

Code from this fantastic article by Andrew Muller https://andrew-muller.medium.com/scraping-steam-user-reviews-9a43f9e38c92.

In [5]:
import requests
r = requests.get(url="https://store.steampowered.com/appreviews/1245620?json=1").json()

In [6]:
from bs4 import BeautifulSoup
def get_app_id(game_name):
  response = requests.get(url=f'https://store.steampowered.com/search/?term={game_name}&category1=998', headers={'User-Agent': 'Mozilla/5.0'})
  soup = BeautifulSoup(response.text, 'html.parser')
  app_id = soup.find(class_='search_result_row')['data-ds-appid']
  return app_id

def get_reviews(appid, params={'json':1}):
  url = 'https://store.steampowered.com/appreviews/'
  response = requests.get(url=url+appid, params=params, headers={'User-Agent': 'Mozilla/5.0'})
  return response.json()
      
def get_n_reviews(appid, n=100, review_type="all", filter="all"):
  reviews = []
  cursor = '*'
  params = {
          'json' : 1,
          'filter' : filter,
          'language' : 'english',
          'day_range' : 9223372036854775807,
          'review_type' : review_type,
          'purchase_type' : 'all'
          }

  while n > 0:
      params['cursor'] = cursor.encode()
      params['num_per_page'] = min(100, n)
      n -= 100

      response = get_reviews(appid, params)
      cursor = response['cursor']
      reviews += response['reviews']

      if len(response['reviews']) < 100: break

  return reviews

# Accessing reviews

In [13]:
import pandas as pd
id = get_app_id("elden ring")
cols = ["review", "voted_up", "received_for_free", "timestamp_created", "timestamp_updated"]
rev = pd.DataFrame(get_n_reviews(id, n=100000, review_type="all"))
# pos = pd.DataFrame(get_n_reviews(id, n=50000, review_type="positive"))[cols]
# neg = pd.DataFrame(get_n_reviews(id, n=50000, review_type="negative", filter="recent"))[cols]

In [8]:
print(pos.shape)
print(neg.shape)

(50000, 5)
(25240, 5)


# Saving reviews

In [14]:
# rev.to_csv("er-reviews.csv", encoding="utf-8", index=False)
# pos.to_csv("er-pos-ts.csv", encoding="utf-8", index=False)
# neg.to_csv("er-neg-ts.csv", encoding="utf-8", index=False)